In [1]:
import pulp
from pulp import *

In [2]:
import numpy as np
import pandas as pd

In [3]:
members = range(1,27)
days = range(1,28)
roles = range(1,18)

In [4]:
# defining the variables
x = pulp.LpVariable.dicts("x",(members,days,roles),0,1,pulp.LpInteger)

In [5]:
# objective
model = pulp.LpProblem("Toastmasters", pulp.LpMaximize)
model += pulp.lpSum(x[si][dj][ri] for si in members for dj in days for ri in roles)

In [6]:
# 17 roles per meeting
for d in days:
    model += pulp.lpSum(x[m][d][r] for m in members for r in roles) == 17

In [7]:
# every member has at least done a role once
for m in members:
    for r in roles:
        model += pulp.lpSum(x[m][d][r] for d in days) >= 1

In [8]:
# only one role per member per meeting
for m in members:
    for d in days:
        model += pulp.lpSum(x[m][d][r] for r in roles) <= 1

In [9]:
# every role in the meeting must be occupied
for d in days:
    for r in roles:
        model += pulp.lpSum(x[m][d][r] for m in members) == 1

In [10]:
# Solve ILP problem and post-processing to get the summary
model.solve()

1

In [11]:
model

Toastmasters:
MAXIMIZE
1*x_10_10_1 + 1*x_10_10_10 + 1*x_10_10_11 + 1*x_10_10_12 + 1*x_10_10_13 + 1*x_10_10_14 + 1*x_10_10_15 + 1*x_10_10_16 + 1*x_10_10_17 + 1*x_10_10_2 + 1*x_10_10_3 + 1*x_10_10_4 + 1*x_10_10_5 + 1*x_10_10_6 + 1*x_10_10_7 + 1*x_10_10_8 + 1*x_10_10_9 + 1*x_10_11_1 + 1*x_10_11_10 + 1*x_10_11_11 + 1*x_10_11_12 + 1*x_10_11_13 + 1*x_10_11_14 + 1*x_10_11_15 + 1*x_10_11_16 + 1*x_10_11_17 + 1*x_10_11_2 + 1*x_10_11_3 + 1*x_10_11_4 + 1*x_10_11_5 + 1*x_10_11_6 + 1*x_10_11_7 + 1*x_10_11_8 + 1*x_10_11_9 + 1*x_10_12_1 + 1*x_10_12_10 + 1*x_10_12_11 + 1*x_10_12_12 + 1*x_10_12_13 + 1*x_10_12_14 + 1*x_10_12_15 + 1*x_10_12_16 + 1*x_10_12_17 + 1*x_10_12_2 + 1*x_10_12_3 + 1*x_10_12_4 + 1*x_10_12_5 + 1*x_10_12_6 + 1*x_10_12_7 + 1*x_10_12_8 + 1*x_10_12_9 + 1*x_10_13_1 + 1*x_10_13_10 + 1*x_10_13_11 + 1*x_10_13_12 + 1*x_10_13_13 + 1*x_10_13_14 + 1*x_10_13_15 + 1*x_10_13_16 + 1*x_10_13_17 + 1*x_10_13_2 + 1*x_10_13_3 + 1*x_10_13_4 + 1*x_10_13_5 + 1*x_10_13_6 + 1*x_10_13_7 + 1*x_10_13_8 + 1*x_10_

In [12]:
for vi in model.variables():
    if vi.varValue == 1:
        print(f"{vi.name} = {vi.varValue:.0f}")

x_10_10_12 = 1
x_10_11_6 = 1
x_10_13_11 = 1
x_10_14_9 = 1
x_10_15_16 = 1
x_10_16_14 = 1
x_10_19_3 = 1
x_10_1_17 = 1
x_10_20_2 = 1
x_10_21_4 = 1
x_10_22_5 = 1
x_10_25_7 = 1
x_10_26_10 = 1
x_10_2_1 = 1
x_10_4_13 = 1
x_10_5_8 = 1
x_10_8_15 = 1
x_10_9_2 = 1
x_11_12_11 = 1
x_11_14_3 = 1
x_11_15_14 = 1
x_11_16_12 = 1
x_11_18_15 = 1
x_11_19_17 = 1
x_11_1_16 = 1
x_11_20_6 = 1
x_11_21_2 = 1
x_11_23_10 = 1
x_11_24_4 = 1
x_11_25_1 = 1
x_11_26_13 = 1
x_11_27_8 = 1
x_11_3_5 = 1
x_11_5_7 = 1
x_11_8_9 = 1
x_11_9_8 = 1
x_12_10_5 = 1
x_12_11_17 = 1
x_12_12_3 = 1
x_12_14_1 = 1
x_12_15_2 = 1
x_12_16_7 = 1
x_12_18_14 = 1
x_12_19_15 = 1
x_12_1_13 = 1
x_12_21_9 = 1
x_12_22_8 = 1
x_12_23_16 = 1
x_12_24_12 = 1
x_12_27_6 = 1
x_12_2_10 = 1
x_12_3_4 = 1
x_12_6_11 = 1
x_13_11_4 = 1
x_13_12_1 = 1
x_13_13_13 = 1
x_13_15_9 = 1
x_13_17_6 = 1
x_13_19_5 = 1
x_13_1_2 = 1
x_13_20_7 = 1
x_13_21_8 = 1
x_13_22_14 = 1
x_13_23_17 = 1
x_13_25_11 = 1
x_13_27_10 = 1
x_13_2_12 = 1
x_13_5_6 = 1
x_13_7_3 = 1
x_13_8_16 = 1
x_13_9_15

## Converting to a dataframe for a clean output

In [13]:
myperson = []
myday = []
myrole = []
value = []

for vi in model.variables():
    person = vi.name.split("_")[1]
    day = vi.name.split("_")[2]
    role = vi.name.split("_")[3]
    
    myperson.append(person)
    myday.append(day)
    myrole.append(role)
    value.append(vi.varValue)
    
data = {'Person':myperson, 'Day':myday, 'Role':myrole, 'Value':value} 

df = pd.DataFrame(data)

    

In [14]:
#df.to_csv('role_optimazation.csv', index = False)

In [15]:
df

,Person,Day,Role,Value
0,10,10,1,0.0
1,10,10,10,0.0
2,10,10,11,0.0
3,10,10,12,1.0
4,10,10,13,0.0
...,...,...,...,...
11929,9,9,5,0.0
11930,9,9,6,0.0
11931,9,9,7,0.0
11932,9,9,8,0.0


In [24]:
roles = [
'Ah Counter', 
'Grammarian',
'Timer', 
'Toastmaster', 
'Meeting speaker 1',
'Meeting speaker 2',
'Meeting speaker 3',
'meeting speaker 4',
'Speech Evaluator 1',
'Speech Evaluator 2',
'Speech Evaluator 3',
'Speech Evaluator 4',
'General evaluator', 
'Table topic master', 
'Table topic evalauator 1',
'Table topic evalauator 2',
'Harkmaster' ]

person = [
    'Member 1',
'Member 2',
'Member 3',
'Member 4',
'Member 5',
'Member 6',
'Member 7',
'Member 8',
'Member 9',
'Member 10',
'Member 11',
'Member 12',
'Member 13',
'Member 14',
'Member 15',
'Member 16',
'Member 17',
'Member 18',
'Member 19',
'Member 20',
'Member 21',
'Member 22',
'Member 23',
'Member 24',
'Member 25',
'Member 26'
]

In [25]:
person_dict = {}
for i in enumerate(person):
    person_dict[i[0] + 1] = i[1]
    
role_dict = {}
for i in enumerate(roles):
    role_dict[i[0] + 1] = i[1]

In [26]:
df['Name'] = df['Person'].apply(lambda x:person_dict[int(x)])
df['Assignment'] = df['Role'].apply(lambda x:role_dict[int(x)])

In [27]:
df

,Person,Day,Role,Value,Name,Assignment,y
0,10,10,1,0.0,Member 10,Ah Counter,0
1,10,10,10,0.0,Member 10,Speech Evaluator 2,0
2,10,10,11,0.0,Member 10,Speech Evaluator 3,0
3,10,10,12,1.0,Member 10,Speech Evaluator 4,12
4,10,10,13,0.0,Member 10,General evaluator,0
...,...,...,...,...,...,...,...
11929,9,9,5,0.0,Member 9,Meeting speaker 1,0
11930,9,9,6,0.0,Member 9,Meeting speaker 2,0
11931,9,9,7,0.0,Member 9,Meeting speaker 3,0
11932,9,9,8,0.0,Member 9,meeting speaker 4,0


In [28]:
def mapper(row):
    if row['Value'] == 1:
        return str(int(row['Role']))
    return str(int(row['Value']))

In [29]:
df['y'] = df.apply(mapper,axis = 1)

In [30]:
df

,Person,Day,Role,Value,Name,Assignment,y
0,10,10,1,0.0,Member 10,Ah Counter,0
1,10,10,10,0.0,Member 10,Speech Evaluator 2,0
2,10,10,11,0.0,Member 10,Speech Evaluator 3,0
3,10,10,12,1.0,Member 10,Speech Evaluator 4,12
4,10,10,13,0.0,Member 10,General evaluator,0
...,...,...,...,...,...,...,...
11929,9,9,5,0.0,Member 9,Meeting speaker 1,0
11930,9,9,6,0.0,Member 9,Meeting speaker 2,0
11931,9,9,7,0.0,Member 9,Meeting speaker 3,0
11932,9,9,8,0.0,Member 9,meeting speaker 4,0


In [32]:
# df.to_csv('role_optimization_final.csv', index = False)